#### konlpy를 사용한 형태소분리와 Word2Vec를 사용한 벡터변환

In [169]:
# # 한국어 전처리를 위한
# !pip install konlpy
# !pip install Twitter
# !pip install tqdm
# !pip install gensim
# !pip install --upgrade jupyter ipywidgets

In [170]:
import os # 운영 체제 관련 작업을 수행하기 위한 Python 표준 라이브러리 중 하나 -> 자바 환경 변수를 설정하기 위해 사용
import pandas as pd # 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 데이터를 데이터프레임으로 로드하고 조작하는 데 사용
import numpy as np # 배열 및 수치 연산을 수행하기 위한 라이브러리 -> 다차원 배열 및 수학적 함수를 다루는 데 사용
from konlpy.tag import Okt # Konlpy 라이브러리에서 제공하는 한국어 형태소 분석기 중 하나 ->  Okt를 초기화하는 데 사용
from gensim.models.word2vec import Word2Vec # Gensim 라이브러리에서 제공하는 Word2Vec 모델을 생성하기 위한 클래스 -> 단어를 고차원 벡터로 표현하여 단어 간 유사성 및 의미 관계를 파악

In [171]:
# 본인 컴퓨터와 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [172]:
# 데이터를 데이터프레임으로 로드
data = pd.read_csv('../cleaned_data.csv', encoding='utf-8')# 데이터 파일 경로
print(data)

       SEQ_NO  ISBN_THIRTEEN_NO VLM_NM                              TITLE_NM  \
0     6352228     9791156759270    NaN   너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이   
1     6352229     9791168120877    NaN  일기에도 거짓말을 쓰는 사람  99년생 시인의 자의식 과잉 에세이   
2     6352230     9791168120839    NaN            본격 한중일 세계사 12  임오군란과 통킹 위기   
3     6352231     9791168120846    NaN   즉시 기분을 바꿔드립니다  신기하게 마음이 편해지는 응급 처방전   
4     6352232     9791168120747    NaN               오늘도 리추얼  음악 나에게 선물하는 시간   
...       ...               ...    ...                                   ...   
1997  6354246     9791164670871    NaN                          학교 서클대화가 필요해   
1998  6354247     9788994229003    NaN                      People Make City   
1999  6354248     9788994027203    NaN                                 동방해경표   
2000  6354249     9791156759126    NaN                      큰글자도서 그렇다면 정상입니다   
2001  6354250     9788988963258    NaN                                 그 사람들   

                  AUTHR_NM PUBLISHER_NM

In [173]:
# 형태소 분리를 위한 Konlpy 객체 초기화
twitter = Okt()

In [188]:
# 각 문서의 형태소 분석을 수행하고, 결과를 리스트에 저장
tokenized_data = data['BOOK_INTRCN_CN'].apply(lambda x: twitter.morphs(str(x)))
print(tokenized_data)

0       [2004년, 방영, 한, 애니메이션, 달빛천사, 에서, 주인공, 루나, 풀문, 역...
1       [그러니, 나, 는, 말, 하고, 싶은, 것, 을, 말, 하겠다, 침착하게, 사랑,...
2       [한중일, 관계, 의, 결정, 적, 분기점, 인, 임오군란, 의, 막전, 막후, 를...
3       [누구, 에게나, 기분, 구급상자, 가, 필요하다, 하나, 씩, 하나, 씩, 차근차...
4       [나다운, 일상, 을, 만드는, 사람, 들, 의, 이야기, 를, 담은, 오늘, 도,...
                              ...                        
1997    [평화로운, 학교, 공동체, 의, 여정, 으로, 가는, 친절한, 안내서, 관계, 를...
1998    [따뜻한, 성북동, 만들기, 피블, 메이크, 시티, 시리즈, 는, 대상, 지역, 의...
1999                                                [nan]
2000    [심야, 치유, 식당, 사랑, 하기에, 결코, 늦지, 않았다, 엄마, 의, 빈틈, ...
2001                                                [nan]
Name: BOOK_INTRCN_CN, Length: 2002, dtype: object


In [189]:
# 데이터프레임의 'BOOK_INTRCN_CN' 열에 NaN 값이 있을 때 빈 문자열로 대체
tokenized_data = tokenized_data.fillna('')
print(tokenized_data)

0       [2004년, 방영, 한, 애니메이션, 달빛천사, 에서, 주인공, 루나, 풀문, 역...
1       [그러니, 나, 는, 말, 하고, 싶은, 것, 을, 말, 하겠다, 침착하게, 사랑,...
2       [한중일, 관계, 의, 결정, 적, 분기점, 인, 임오군란, 의, 막전, 막후, 를...
3       [누구, 에게나, 기분, 구급상자, 가, 필요하다, 하나, 씩, 하나, 씩, 차근차...
4       [나다운, 일상, 을, 만드는, 사람, 들, 의, 이야기, 를, 담은, 오늘, 도,...
                              ...                        
1997    [평화로운, 학교, 공동체, 의, 여정, 으로, 가는, 친절한, 안내서, 관계, 를...
1998    [따뜻한, 성북동, 만들기, 피블, 메이크, 시티, 시리즈, 는, 대상, 지역, 의...
1999                                                [nan]
2000    [심야, 치유, 식당, 사랑, 하기에, 결코, 늦지, 않았다, 엄마, 의, 빈틈, ...
2001                                                [nan]
Name: BOOK_INTRCN_CN, Length: 2002, dtype: object


In [ ]:
--- 

In [176]:
# Word2Vec 모델 훈련을 위한 데이터 준비
sentences = data['BOOK_INTRCN_CN'].tolist()

#### train과 test dataset 8:2로 나누기

In [177]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [178]:
# 데이터를 학습 데이터와 테스트 데이터로 분리
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [179]:
# 'BOOK_INTRCN_CN' 열을 선택 후 apply를 사용해 데이터프레임의 각 행에 함수를 적용
# str(x)는 각 문서를 문자열로 변환하고 twitter.morphs() 함수는 주어진 문자열을 형태소로 분리 -> 텍스트를 형태소로 분리하는 과정이다.
# 그 후 .tolist()메서드를 사용해 형태소 분리된 결과를 리스트로 변환되는데 각 문서가 형태소로 분리된 리스트로 저장된다.
# 학습 데이터용 Word2Vec 모델 훈련
sentences_train = train_data['BOOK_INTRCN_CN'].apply(lambda x: twitter.morphs(str(x))).tolist()
model_train = Word2Vec(sentences=sentences_train, vector_size=100, window=5, min_count=5, sg=0)

In [180]:
# 테스트 데이터용 Word2Vec 모델 훈련
sentences_test = test_data['BOOK_INTRCN_CN'].apply(lambda x: twitter.morphs(str(x))).tolist()
model_test = Word2Vec(sentences=sentences_test, vector_size=30, window=5, min_count=5, sg=0)

In [181]:
# 각 문서의 벡터 생성
def get_doc_vector(doc):
    word_vectors = [model.wv[word] for word in doc if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    doc_vector = np.mean(word_vectors, axis=0)
    return doc_vector

In [182]:
# 각 문서의 벡터 생성 (학습 데이터 및 테스트 데이터에 대해 각각)
train_data['doc_vector'] = train_data['BOOK_INTRCN_CN'].apply(lambda x: get_doc_vector(twitter.morphs(str(x), model)))
test_data['doc_vector'] = test_data['BOOK_INTRCN_CN'].apply(lambda x: get_doc_vector(twitter.morphs(str(x), model)))

#### 유사도 검사 -코사인 유사도

In [183]:
# 유사도 계산 (학습 데이터와 테스트 데이터 간)
cosine_sim_train = cosine_similarity(np.stack(train_data['doc_vector']), np.stack(train_data['doc_vector']))
cosine_sim_test = cosine_similarity(np.stack(test_data['doc_vector']), np.stack(train_data['doc_vector']))
print("cosine_sim_train", cosine_sim_train)
print()
print("cosine_sim_test",cosine_sim_test)

cosine_sim_train [[0.99999994 0.99999994 0.99999994 ... 0.99999994 0.99999994 0.14026953]
 [0.99999994 0.99999994 0.99999994 ... 0.99999994 0.99999994 0.14026953]
 [0.99999994 0.99999994 0.99999994 ... 0.99999994 0.99999994 0.14026953]
 ...
 [0.99999994 0.99999994 0.99999994 ... 0.99999994 0.99999994 0.14026953]
 [0.99999994 0.99999994 0.99999994 ... 0.99999994 0.99999994 0.14026953]
 [0.14026953 0.14026953 0.14026953 ... 0.14026953 0.14026953 1.0000001 ]]

cosine_sim_test [[0.99999994 0.99999994 0.99999994 ... 0.99999994 0.99999994 0.14026953]
 [0.14027233 0.14027233 0.14027233 ... 0.14027233 0.14027233 0.99918824]
 [0.13932896 0.13932896 0.13932896 ... 0.13932896 0.13932896 0.999205  ]
 ...
 [0.99999994 0.99999994 0.99999994 ... 0.99999994 0.99999994 0.14026953]
 [0.13739099 0.13739099 0.13739099 ... 0.13739099 0.13739099 0.99917847]
 [0.1388085  0.1388085  0.1388085  ... 0.1388085  0.1388085  0.9992235 ]]


In [184]:
# 유사성을 확인할 문서의 인덱스 선택 (예시: 첫 번째 문서)
document_index = 100

In [185]:
similarities = cosine_sim_test[document_index]  # 해당 테스트 문서와 학습 데이터 간의 유사도
similarities

array([0.99999994, 0.99999994, 0.99999994, ..., 0.99999994, 0.99999994,
       0.14026953], dtype=float32)

In [186]:
# 유사도를 기준으로 내림차순으로 정렬한 후 상위 5개를 추출
indices = np.argsort(similarities)[::-1][:5]
sorted_similarities = similarities[indices]

In [187]:
# 가장 유사한 문서 5개 출력
for i, similar_document_index in enumerate(indices):
    similarity = sorted_similarities[i]
    similar_title = train_data.loc[similar_document_index, 'TITLE_NM']
    similar_doc = train_data.loc[similar_document_index, 'BOOK_INTRCN_CN']
    print(f"제목 #{i + 1}: {similar_title}")
    print(f"줄거리: {similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

제목 #1: 너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이
줄거리: 2004년 방영한 애니메이션 달빛천사에서 주인공 루나풀문 역을 맡으며 90년대생들에게 보석 같은 추억을 선물한 성우 이용신의 첫 번째 에세이 수많은 작품의 주연을 맡으며 쉬지 않고 대중에게 행복을 전해온 성우 이용신의 발자취를 확인할 수 있다
유사도: 1.0000

제목 #2: 불의 날개와 비밀의 왕국  상
줄거리: 전 세계 21개국 출간 뉴욕타임스 베스트셀러 1위  1000만 부 이상 판매 불의 날개 시리즈 제3부 세 번째 이야기에서는 자신의 정체성과 가치를 찾기 위해 정글 부족으로 찾아간 글로리의 이야기가 펼쳐진다
유사도: 1.0000

제목 #3: 불의 날개와 비밀의 왕국  하
줄거리: 독자들을 판타지 세상 파이리아에 푹 빠져들게 할 블록버스터급 대형 판타지 불의 날개 의 제 3부가 출간되었다 세 번째 이야기에서는 자신의 정체성과 가치를 찾기 위해 정글 부족으로 찾아간 글로리의 이야기가 펼쳐진다
유사도: 1.0000

제목 #4: 1일無식
줄거리: 수년간의 과학 연구와 임상 실험 그리고 흥미진진한 이야기들을 통해 우리 몸의 작용 원리와 자기 치유 능력 그리고 그 원리를 돕는 방법들을 소개하고 각종 단식법에 의한 질병 치유의 과정을 알기 쉽게 밝힌다
유사도: 1.0000

제목 #5: 아무것도 하지 않는 법
줄거리: 아무것도 하지 않는 것은 휴대폰을 내려놓고 그 자리에 가만히 머무는 것이다 아무것도 하지 않는 법의 저자 제니 오델은 소셜미디어를 비롯한 관심경제에 사로잡힌 관심의 주권을 되찾아 다른 방향으로 확장하자고 제안한다
유사도: 1.0000

